# Demos: Lecture 20

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

## Demo 1: phase flip code

In [ ]:
def prepare_state(wires):
    #qml.RY(np.pi/3, wires=wires)
    qml.Hadamard(wires=0)
    #qml.PauliX(wires=0)

def encode_phase_flip(wires):
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.CNOT(wires=[wires[0], wires[2]])
    for wire in wires:
        qml.Hadamard(wires=wire)

def detect_and_recover_phase_flip(wires):
    for wire in wires:
        qml.Hadamard(wires=wire)
    qml.CNOT(wires=[wires[0], wires[2]])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.Toffoli(wires=wires[::-1])

dev = qml.device("default.mixed", wires=3)

@qml.qnode(dev)
def phase_flip_code(phase_to_flip=None):
    prepare_state(wires=0)
    encode_phase_flip(wires=dev.wires)
    if phase_to_flip is not None:
        qml.PauliZ(wires=phase_to_flip)
    detect_and_recover_phase_flip(dev.wires)
    return qml.state()

In [ ]:
qml.draw_mpl(phase_flip_code)(0)

In [ ]:
qml.math.partial_trace(phase_flip_code(0), indices=[1, 2])

## Demo 2: Shor 9-qubit code

In [ ]:
def prepare_state(wires):
    qml.RY(np.pi/3, wires=wires)
    #qml.Hadamard(wires=0)


def encode_bit_flip(wires):
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.CNOT(wires=[wires[0], wires[2]])

def detect_and_recover_bit_flip(wires):
    qml.CNOT(wires=[wires[0], wires[2]])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.Toffoli(wires=wires[::-1])


def encode_shor(wires):
    encode_phase_flip([wires[0], wires[3], wires[6]])
    qml.Barrier()
    encode_bit_flip(wires[:3])
    encode_bit_flip(wires[3:6])
    encode_bit_flip(wires[6:])

def detect_and_recover_shor(wires):
    detect_and_recover_bit_flip(wires[:3])
    detect_and_recover_bit_flip(wires[3:6])
    detect_and_recover_bit_flip(wires[6:])
    qml.Barrier()
    detect_and_recover_phase_flip([wires[0], wires[3], wires[6]])

dev = qml.device("default.mixed", wires=9)

@qml.qnode(dev)
def shor_code(bit_to_flip=None, phase_to_flip=None):
    prepare_state(wires=0)
    
    encode_shor(wires=dev.wires)
    qml.Barrier()
    # Errors
    if bit_to_flip is not None:
        qml.PauliX(wires=bit_to_flip)
    if phase_to_flip is not None:
        qml.PauliZ(wires=phase_to_flip)
    qml.Barrier()
    
    detect_and_recover_shor(dev.wires)
    return qml.state()

In [ ]:
qml.draw_mpl(shor_code)(6, 6)

In [ ]:
qml.math.partial_trace(shor_code(4, 5), indices=list(range(1, 9)))

## Demo 3: bit flip code with stabilizers

In [ ]:
def prepare_state(wires):
    #qml.RY(np.pi/3, wires=wires)
    qml.Identity(wires=0)
    
def encode_bit_flip(wires):
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.CNOT(wires=[wires[0], wires[2]])

def measure_stabilizers(wires):
    qml.Hadamard(wires=3)
    qml.CZ(wires=[0, 3])
    qml.CZ(wires=[1, 3])
    qml.Hadamard(wires=3)
    
    qml.Hadamard(wires=4)
    qml.CZ(wires=[1, 4])
    qml.CZ(wires=[2, 4])
    qml.Hadamard(wires=4)

dev = qml.device("default.mixed", wires=5)

@qml.qnode(dev)
def bit_flip_code(bit_to_flip=None):
    prepare_state(wires=0)
    encode_bit_flip(wires=dev.wires)
    qml.Barrier()
    if bit_to_flip is not None:
        qml.PauliX(wires=bit_to_flip)
    qml.Barrier()
    measure_stabilizers(dev.wires)
    return qml.expval(qml.PauliZ(3)), qml.expval(qml.PauliZ(4))

In [ ]:
qml.draw_mpl(bit_flip_code)(1)

In [ ]:
bit_flip_code(2)